# Interactive NDVI

In [2]:
import datetime
import geemap
import ee
from astral import LocationInfo
from astral.sun import sun

# Band aliases.
BLUE = 'CMI_C01'
RED = 'CMI_C02'
VEGGIE = 'CMI_C03'

bands = [BLUE, RED, VEGGIE]

goes_collection = ee.ImageCollection('NOAA/GOES/16/MCMIPC') 

def set_coordinates(left, bottom, right, top):
    return ee.Geometry.Bbox(left, bottom, right, top)


def applyScaleandOffset(image):
    
    band_list = image.select('CMI_C..').bandNames()

    scales = (bandlist.map(lambda name: ee.String(name).cat('_scale'))).map(lambda band: image.getNumber(band))
    offsets = (bandlist.map(lambda name: ee.String(name).cat('_offset'))).map(lambda band: image.getNumber(band))

    scaled = image.select(band_list).multiply(scales).add(offsets).rename(band_list)

    return image.addBands(srcImg = scaled, overwrite = True)


def get_daytime(lat, lon):

    loc = LocationInfo(latitude=lat, longitude=lon)
    s = sun(loc.observer, date=datetime.date.today())


In [ ]:
roi = set_coordinates(-102.84, 37.25, -96.91, 42.27)



# Filter the GOES collection to a specific time range.
yesterday = ee.Date.now().advance(-1, 'day')

In [ ]:
filtered = 

In [ ]:

m = geemap.Map()
m